# 特異クラス

* 特異クラス
    * 指定したインスタンスだけに適用される特別なクラス
        * 無名クラス(モジュールがインクルードされたときにそのメソッドが属するクラス)と同様にインタプリタ内部で使用するクラス
            * ユーザに意識させたくないクラス
        * 無名クラスと混同しないように気を付ける
    * Silver試験の出題範囲外

## 特異クラスの性質

* 特定のインスタンスに特別な性質を持たせたい
    * 継承したクラスを書いて新たにインスタンス化するほどではない
    * いちいちサブクラスを定義するのは面倒くさい
        * このときは特異メソッドを使う
* 特異メソッド
    * インスタンスに直接定義したメソッド

In [13]:
class Foo
end
foo1 = Foo.new
foo2 = Foo.new

### インスタンスに直接メソッドを定義 (特異メソッドの定義)
def foo1.methodA
  1 + 1
end
puts foo1.methodA
puts "foo1のクラスはFooのまま"
puts foo1.class

puts "foo2にmethodAは存在しない"
puts foo2.methodA


2
foo1のクラスはFooのまま
Foo
foo2にmethodAは存在しない


NoMethodError: undefined method `methodA' for #<Foo:0x007f083f08df88>

* 仕組み
    1. 特異メソッドを定義する
    1. Rubyインタプリタは特異クラスと呼ばれるクラスを作成した継承チェーンに組み込む
        * foo1の特異クラスは「#foo1」と表現する
        * 特異クラス#foo1のsuperclassはFooクラス (#foo < Foo のようなイメージ)
        * foo1のクラスメソッドはFooクラスを指しているまま

## 特異クラスの参照

* 特異クラスは通常ユーザに意識させたくないものであるが、参照は可能

特異クラスの再オープン式
``` ruby
class << (対象オブジェクト)
end
```

In [25]:
class Foo
end
foo1 = Foo.new

SingletonClassA = class << foo1
  self
end

puts SingletonClassA
puts SingletonClassA.superclass

#<Class:#<Foo:0x007f083f15a088>>
Foo


## 特異クラスの再オープンとメソッドの定義

` def <オブジェクト名>.<新たに定義するメソッド名> ... end ` でなくても特異クラスの再オープン式でメソッドを定義できる

In [28]:
class Foo
end
foo1 = Foo.new
foo2 = Foo.new

### defを使った特異メソッドの定義
def foo1.methodA
  1 + 1
end

### 特異クラスの再オープンと特異メソッド定義
class << foo2
  def methodB
    2 + 2
  end
end

puts foo1.methodA
puts foo2.methodB

2
4


## selfとメソッドの定義先

* self
    * 自分自身を示す特別なオブジェクト
    * カレントオブジェクトとも呼ばれる
        * カレントオブジェクトはselfキーワードでアクセスできる
        * Rubyのコードはカレントオブジェクト内部で実行される
    * selfの役割を担うオブジェクトは同時に複数存在しない
        * 長時間その役割を担うオブジェクトも存在しない
    * どこで使われるかによって示すオブジェクトが異なる
        * クラス内部
            * そのクラスを保持
        * メソッド内部
            * そのメソッドを実行するオブジェクトを保持
            * メソッドを呼び出すときはレシーバがselfになる
                * その時点で全てのインスタンス変数はselfのインスタンス変数になる
            * レシーバを明示せずにメソッドを呼び出すと、全てselfに対するメソッド呼び出しになる
                * 他のオブジェクトを明示してメソッドを呼び出すとそのオブジェクトがselfになる
* レシーバ
    * メソッドが実行されるオブジェクトのこと
    * 「メソッドの受け手」という意味

In [47]:
class C1
  self
  def methodD
    self
  end
end

c1 = C1.new
puts "クラス内部で呼び出したself。クラスそのもの"
puts c1
puts "クラスの中のインスタンスメソッド内部で呼び出したself(レシーバ)"
puts c1.methodD
puts "クラスのオブジェクトとレシーバは同じオブジェクト(そのクラスを参照している)"
puts c1 == c1.methodD

クラス内部で呼び出したself
#<C1:0x007f083e836588>
クラスの中のインスタンスメソッド内部で呼び出したself(レシーバ)
#<C1:0x007f083e836588>
クラスのオブジェクトとレシーバは同じオブジェクト(そのクラスを参照している)
true


In [2]:
class ClassA
  self   ### ClassAそのもの
end

ClassA

In [14]:
class ClassB
  def methodB
    self   ### methodB
  end
end 

:methodB

In [1]:
class MyClass
  def testing_self
    @var = 10          ### selfのインスタンス変数
    my_method()　　　### self.my_method()と同じ
    self                     ### selfへの参照を返す
  end
  
  def my_method
    @var = @var + 1
  end
end

obj = MyClass.new
obj.testing_self   ### この瞬間objがselfになる   @varはobjのインスタンス変数になり、my_methods()メソッドが呼び出されるのもobjになる

#<MyClass:0x007fa1e128b1f0 @var=11>

### mainオブジェクト

* メソッドを呼び出さない時のselfがmainオブジェクト
    * トップレベルコンテキストとも呼ばれる
            * コールスタックがトップレベルのときに居るオブジェクトだから
* トップレベルとはどんなときか
    * メソッドを呼び出していないときの状態
    * 呼び出したメソッドが全て帰ってきたときの状態

### privateについて

* 4-6.MethodVisualization.ipynb
* 5-1.ObjectClass.ipynb


## メソッドのネスト

* メソッドは定義したクラスに所属する
* メソッドがネストした場合、内側のメソッドは外側のメソッドが定義されたクラスに定義される
    * ただし、<font color="red">外側のメソッドが実行されるまでは定義されない</font>


In [46]:
class C3
  def methodE
    def methodF
    end
  end
end

puts "まだmethodFは定義されていない"
puts C3.instance_methods(false)
C3.new.methodE
puts "methodEが実行されたためmethodFが定義された"
puts C3.instance_methods(false)


まだmethodFは定義されていない
[:methodE]
methodEが実行されたためmethodFが定義された
[:methodE, :methodF]


## 特異クラスに対するMix-in

* include
    * クラスにモジュールをincludeする
* extend
    * 特異クラスへのMix-inで良く使用する
    * includeよりシンプルに書ける
* prepend
    * includeと同様にモジュールのメソッドをクラスに取り込むためのメソッド
    * クラスよりも先にモジュールのメソッド探索を行う
    * 優先度
        * prepend > (クラス内のメソッド) > include

In [67]:
module Bar
  def methodA
    3+4
  end
end

class Foo
end

foo1 = Foo.new
foo2 = Foo.new

### includeを使ったMix-in
class << foo1
  include Bar
end
puts foo1.methodA

### extendメソッドを使ったMix-in
### foo2インスタンスでモジュールBarのメソッドmethodAを実行
puts foo2.extend(Bar).methodA

7
7


In [77]:
module M1
  def method1; "module M1 method1"; end
end

class C1
  prepend M1
  def method1; "class C1 method1"; end
end

class C2
  def method1; "class C2 method1"; end
  include M1
end

puts "prependはクラス内の度名のメソッドよりも優先される"
puts C1.new.method1

puts "includeはクラス内に同名のメソッドがあればそちらを実行する"
puts C2.new.method1

puts "ancestorsの表示順の違い"
puts C1.ancestors
puts C2.ancestors

prependはクラス内の度名のメソッドよりも優先される
module M1 method1
includeはクラス内に同名のメソッドがあればそちらを実行する
class C2 method1
ancestorsの表示順の違い
[M1, C1, Object, PP::ObjectMixin, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]
[C2, M1, Object, PP::ObjectMixin, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]


## prepend内でsuperを呼び出す

1. クラスC1内でprependでモジュールM1を読み込む
1. M1内のsuperでスーパークラス(C1)を呼び出す
1. C1のmethod1が実行される
1. 続いてM1モジュール内のputsが実行される

includeの場合はスーパークラスがObjectになり、Objectクラスにmethod1は存在しないためsuperでは何も実行されず、C1インスタンスメソッドmethod1のみ実行される

In [84]:
module M1
  def method1
    super
    puts "module M1"
  end
end

class C1
  prepend M1
  #include M1
  def method1
    puts "class C1"
  end
end

puts C1.ancestors
C1.new.method1

[M1, C1, Object, PP::ObjectMixin, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]
class C1
module M1


## Refinements

* Refinements
    * クラスに対するメソッドの追加や変更が影響する範囲を絞れる
    * Ruby2.1以降の機能
    * メソッド
        * refineメソッド
            * 変更を加えるクラスを宣言
        * usingメソッド
            * 以降変更したメソッドの呼び出しが有効になる
    * usingが有効になる範囲
        * クラス・モジュールの定義の外で呼び出し
            * ファイルの末尾まで
        * クラス・モジュールの定義の中で呼び出し
            * そのクラス・モジュール定義の末尾まで
* 以下のプログラムの流れ
    1. ClassAのfooメソッドが定義される
        * `puts "ClassA#foo"`
    1. ClassAのbarメソッドが定義される
        * `foo`
        * すなわち `puts "ClassA#foo"`
    1. using ModuleAにより、以降ModuleAが有効になる
        * fooメソッドはModuleAのfooメソッド `puts "ClassA#foo in module ModuleA"` になる
    1. xにClassAのインスタンスが生成される
    1. x.fooが実行される
        * `ClassA#foo in module ModuleA`
    1. x.barが実行される
        * メソッドbar内でメソッドfooが実行される
        * メソッドbarが定義されたのはusing ModuleAの前
        * `ClassA#foo`

In [2]:
class ClassA
  def foo
    puts "ClassA#foo"
  end
  
  def bar
    foo
  end
end

module ModuleA
  refine ClassA do
    def foo
      puts "ClassA#foo in module ModuleA"
    end
  end
end

### ここより下のClassAのfooメソッドはrefineにより書き換えられる
using ModuleA

x = ClassA.new
x.foo
x.bar
puts ClassA.ancestors

ClassA#foo in module ModuleA
ClassA#foo
[ClassA, Object, PP::ObjectMixin, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]
